In [ ]:
#default_exp prodigy_demo

In [ ]:
#no_test

#data structure imports
import pandas as pd
import numpy as np

#python imports
import random

#modeling imports
from spacy.util import fix_random_seed
from ssda_nlp.collate import *
from ssda_nlp.split_data import *
from ssda_nlp.modeling import *
from ssda_nlp.preprocessing import *
from ssda_nlp.model_performance_utils import *

In [ ]:
#no_test
seed = 2436
random.seed(seed)
fix_random_seed(seed)

# First Book of Baptisms, Matanzas (15834)

In [ ]:
#no_test

collated_df = prodigy_output_to_collated_df("transcriptions//15834_annot.jsonl")

# Split data
train_df, valid_df, test_df = split_data_grp(collated_df, prop_train = 0.7, prop_validation = 0.2, grp_var = 'entry_no', seed=seed)

# Generate Spacy datasets
train_spacy = genSpaCyInput(train_df)

# Look at column names
train_df.head(10)

,entry_no,text,entity,start,end,label
13,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Paula,10,15,PER
14,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Esc.a,17,22,CHAR
15,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Juebes veinte y tres de feb.o de mil sietec.to...,23,86,DATE
16,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Thomas de Orvera,90,106,PER
17,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Paula,145,150,PER
18,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,h. l.16,151,158,CHAR
19,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Juan Joseph,162,173,PER
20,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Maria Josepha,177,190,PER
21,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,esc.s,191,196,CHAR
22,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Capitan D. Luis Hurtado de Mendoza,201,235,PER


In [ ]:
#no_test

train_spacy[:2]

[('[margin]: Paula. Esc.a Juebes veinte y tres de feb.o de mil sietec.tos. y diez y nueve Yo Thomas de Orvera baptizé, y pusse los santos15 oleos á Paula h. l.16 de Juan Joseph, y Maria Josepha esc.s del Capitan D. Luis Hurtado de Mendoza fue su Padrino Bartholome Rixo, y lo firmé. [signed]: Thomas de Orvera ',
  {'entities': [(10, 15, 'PER'),
    (17, 22, 'CHAR'),
    (23, 86, 'DATE'),
    (90, 106, 'PER'),
    (145, 150, 'PER'),
    (151, 158, 'CHAR'),
    (162, 173, 'PER'),
    (177, 190, 'PER'),
    (191, 196, 'CHAR'),
    (201, 235, 'PER'),
    (243, 250, 'REL'),
    (251, 266, 'PER'),
    (290, 306, 'PER')]}),
 ('[margin]: Bernardo Esc.vo Domingo nueve de Abril de mil sietectos y diez y nueve Yo Thomas de Orvera baptize, y pusse los s.tos oleos á Bernardo negro adulto de nacion carabali esc.o de D. Juan Joseph de Justis fue su P. Andres de Morales, y lo firmé. [signed]: Thomas de Orvera ',
  {'entities': [(10, 18, 'PER'),
    (19, 25, 'CHAR'),
    (26, 80, 'DATE'),
    (84, 100, 

In [ ]:
#no_test

#if you want the same model everytime:
#random.seed(seed)
#fix_random_seed(seed)

#spacy parameters
cping = {'start':16, 'end':32, 'cp_rate':1.05}
solves = {'learn_rate':0.001}
dpout = 0.6
save_mdir = 'models/15834'

#load model from pretrained
nlp_model = load_model()

nlp_model, perf_df = model_meta_training(nlp_model, train_spacy, valid_df, verbose=True, save_dir = save_mdir,
                                         n_iter=10, solver_params=solves, compound_params=cping, dropout=dpout)

c:\users\genkindn\appdata\local\programs\python\python39\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "[margin]: Fran.co esc.o Domingo dos de Marco de mi..." with entities "[(10, 17, 'PER'), (18, 23, 'CHAR'), (24, 78, 'DATE...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,1.0,3806.833701,1.960784,0.453515,0.736648


Saved model to models\15834


c:\users\genkindn\appdata\local\programs\python\python39\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "[margin]: Fran.co esc.o Domingo dos de Marco de mi..." with entities "[(10, 17, 'PER'), (18, 23, 'CHAR'), (24, 78, 'DATE...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,2.0,1935.017072,51.404494,41.496599,45.922208


...

Saved model to models\15834


c:\users\genkindn\appdata\local\programs\python\python39\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "[margin]: Fran.co esc.o Domingo dos de Marco de mi..." with entities "[(10, 17, 'PER'), (18, 23, 'CHAR'), (24, 78, 'DATE...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,9.0,324.806369,90.675991,88.208617,89.425287


Saved model to models\15834


c:\users\genkindn\appdata\local\programs\python\python39\lib\site-packages\spacy\training\iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "[margin]: Fran.co esc.o Domingo dos de Marco de mi..." with entities "[(10, 17, 'PER'), (18, 23, 'CHAR'), (24, 78, 'DATE...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


# Second Book of Baptisms, St. Augustine (239746)

In [ ]:
#no_test

collated_df = prodigy_output_to_collated_df("transcriptions/239746_annot.jsonl")

# Split data
train_df, valid_df, test_df = split_data_grp(collated_df, prop_train = 0.7, prop_validation = 0.2, grp_var = 'entry_no', seed=seed)

# Generate Spacy datasets
train_spacy = genSpaCyInput(train_df)

# Look at column names
train_df.head(10)

,entry_no,text,entity,start,end,label
31,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Antonio Guillo,3,17,PER
32,2,"2. Antonio Guillo Miercoles, día veinte de Nov...","Miercoles, día veinte de Noviembre de Mil Sete...",18,86,DATE
33,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Don Miguel o’Reilly,90,109,PER
34,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Teniente de Cura Beneficiado,110,138,CHAR
35,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Vicario,140,147,CHAR
36,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Juez Eclesiás tico Auxiliar,151,178,CHAR
37,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Iglesia parroquial,187,205,LOC
38,2,"2. Antonio Guillo Miercoles, día veinte de Nov...","provincia, de San Agustín",208,233,LOC
39,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",Florida Oriental,240,256,LOC
40,2,"2. Antonio Guillo Miercoles, día veinte de Nov...",niño,295,299,CHAR


In [ ]:
#no_test

train_spacy[:2]

[('2. Antonio Guillo Miercoles, día veinte de Noviembre de Mil Setecientos Noventa y Tres Yo Don Miguel o’Reilly Teniente de Cura Beneficiado, Vicario, y Juez Eclesiás tico Auxiliar de esta Iglesia parroquial y provincia, de San Agustín de la Florida Oriental. Baptisé y puse los santos oleos a un niño de color, moreno, que nacio el día primero de octubre del presente año, hijo na tural de los negros libres Isaac Guillo y Sara Ca mel, ambos naturales de America del Norte; y y en el exerci las sacras Ceremonias, y preces de nuestra Santa madre Iglesia, y la puse por nombre Juan Antonio. Fueron sus padrinos Antonio Pellice, y Susana Pellicer vecinos de esta Ciudad, les advertí el parentesco espiritual, y demás obligaciones, y lo firmé en dicho dia, mes y año.  Miguel OReilly ',
  {'entities': [(3, 17, 'PER'),
    (18, 86, 'DATE'),
    (90, 109, 'PER'),
    (110, 138, 'CHAR'),
    (140, 147, 'CHAR'),
    (151, 178, 'CHAR'),
    (187, 205, 'LOC'),
    (208, 233, 'LOC'),
    (240, 256, 'LOC'

In [ ]:
#no_test

#if you want the same model everytime:
#random.seed(seed)
#fix_random_seed(seed)

#spacy parameters
cping = {'start':16, 'end':32, 'cp_rate':1.05}
solves = {'learn_rate':0.001}
dpout = 0.6
save_mdir = 'models/239746'

#load model from pretrained
nlp_model = load_model('es_core_news_md')

nlp_model, perf_df = model_meta_training(nlp_model, train_spacy, valid_df, verbose=True, save_dir = save_mdir,
                                         n_iter=10, solver_params=solves, compound_params=cping, dropout=dpout)

,cycle_no,avg_cycle_loss,precision,recall,f_score
0,1.0,3058.593277,0.673077,0.322284,0.435866


Saved model to models\239746


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,2.0,1895.333521,0.768,0.530387,0.627451


Saved model to models\239746


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,3.0,1568.612363,0.838403,0.812155,0.82507


...

Performance hasn't improved for 3 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,28.0,287.60354,0.890741,0.88582,0.888273


Performance hasn't improved for 4 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,29.0,277.603007,0.876838,0.878453,0.877645


Performance hasn't improved for 5 cycles...
Done training after 29 meta cycles.


It's interesting how similar this training performance was to the Matanzas volume above. If this is replicated across more data sets, it probably means something (although I'm not sure what exactly).

In [ ]:
#no_test

collated_df = prodigy_output_to_collated_df("transcriptions/166470_annot.jsonl")

# Split data
train_df, valid_df, test_df = split_data_grp(collated_df, prop_train = 0.7, prop_validation = 0.2, grp_var = 'entry_no', seed=seed)

# Generate Spacy datasets
train_spacy = genSpaCyInput(collated_df)

# Look at column names
train_spacy[:5]

[('No. 1 Roque Cruz En catorce de Agosto de mil ochocientos treintaynuebe: yo Don Antonio Loreto Sanchez Presbitero cura beneficiado por Su Majestad de la Iglesia Parroquial de la Purisima concepcion de esta villa de Cienfuegos en ella y su jurisdiccion Vicario Eclesiastico por Santa Eclesiastica Illustrisima bautice solemnemente y puse los santos oleos a un negro adulto natural de Guinea de la propiedad de Don Agustin de Santa Cruz: en cuyo negro ejerci las sacras ceremonias y preces y le puse por nombre Roque fueron sus padrinos Jose y Monserrate; aquien adverti el parentesco espiritual y demas obligaciones y lo firme Antonio L. Sanchez ',
  {'entities': [(6, 16, 'PER'),
    (20, 70, 'DATE'),
    (75, 101, 'PER'),
    (113, 117, 'CHAR'),
    (149, 196, 'LOC'),
    (205, 224, 'LOC'),
    (251, 271, 'CHAR'),
    (358, 363, 'CHAR'),
    (364, 370, 'CHAR'),
    (371, 378, 'CHAR'),
    (382, 388, 'LOC'),
    (395, 404, 'CHAR'),
    (408, 433, 'PER'),
    (443, 448, 'CHAR'),
    (508, 513,

In [ ]:
#no_test

#if you want the same model everytime:
#random.seed(seed)
#fix_random_seed(seed)

#spacy parameters
cping = {'start':16, 'end':32, 'cp_rate':1.05}
solves = {'learn_rate':0.001}
dpout = 0.6
save_mdir = 'models/166470'

#load model from pretrained
nlp_model = load_model()

nlp_model, perf_df = model_meta_training(nlp_model, train_spacy, valid_df, verbose=True, save_dir = save_mdir,
                                         n_iter=10, solver_params=solves, compound_params=cping, dropout=dpout)

,cycle_no,avg_cycle_loss,precision,recall,f_score
0,1.0,1523.910855,0.0,0.0,0.0


Saved model to models\166470


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,2.0,739.172972,0.0,0.0,0.0


Performance hasn't improved for 1 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,3.0,728.802224,28.04878,24.731183,26.285714


...

Performance hasn't improved for 3 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,26.0,42.008834,96.808511,97.849462,97.326203


Performance hasn't improved for 4 cycles...


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,27.0,44.406255,97.849462,97.849462,97.849462


Performance hasn't improved for 5 cycles...
Done training after 27 meta cycles.
